# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [3]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [4]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [5]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [6]:
llm = ChatOllama(model = "zephyr",temperature = 0) # format = 'json' is also an argument 

In [7]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

`chain.run()` will format the prompt under the hood and pass it to the LLM model for response

In [8]:
chain = LLMChain(llm=llm, prompt=prompt)

In [10]:
product = "Queen Size Sheet Set"
chain.invoke(product)

{'product': 'Queen Size Sheet Set',
 'text': 'A suitable name for a company that specializes in making queen size sheet sets could be "Regal Bedding" or "Monarch Linens". Other options could include "Royal Comfort Bedding", "Queen\'s Choice Linens", or "Grande Sheet Company". These names convey the idea of luxury, comfort, and size specificity, which would appeal to customers looking for high-quality queen size bedding.'}

## SimpleSequentialChain

In [11]:
from langchain.chains import SimpleSequentialChain

In [12]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [13]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [14]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [15]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
A suitable name for a company that specializes in making queen size sheet sets could be "Regal Bedding" or "Monarch Linens". Other options could include "Royal Comfort Bedding", "Queen's Choice Linens", or "Grande Sheet Company". These names convey the idea of luxury, comfort, and size specificity, which would appeal to customers looking for high-quality queen size bedding.
Indulge in royal comfort with our Regal Bedding or Monarch Linens, featuring luxurious queen size sheet sets that exude sophistication and size specificity. Upgrade your sleep experience with Queen's Choice Linens or Grande Sheet Company, where every thread promises unparalleled comfort and quality.

> Finished chain.


"Indulge in royal comfort with our Regal Bedding or Monarch Linens, featuring luxurious queen size sheet sets that exude sophistication and size specificity. Upgrade your sleep experience with Queen's Choice Linens or Grande Sheet Company, where every thread promises unparalleled comfort and quality."

## SequentialChain

In [16]:
from langchain.chains import SequentialChain
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model = "zephyr",temperature = 0) # format = 'json' is also an argument 

In [17]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [18]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [19]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [20]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [21]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [22]:
review = df.Review[5]
overall_chain.invoke(review)

c:\Users\kaila\DLAI\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The mousse doesn't hold up well, which is strange. I buy the same ones from a store and their taste is much better...\nOld batch or counterfeit!?",
 'summary': 'The reviewer expresses disappointment with the mediocre taste of the product and notes that the mousse does not hold up well, which is unusual as they have had better experiences with the same product from a different store. The reviewer suggests that it could be an old batch or potentially a counterfeit product.',
 'followup_message': "Réponse suivante :\n\nNous sommes désolés d'apprendre que vous soyez déçus du goût médiocre de notre produit et constatons avec regret que la mousse ne tient pas bien, ce qui est étrange puisque nous avons déjà eu des expériences meilleures avec le même produit dans un autre

## Router Chain

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [ ]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

`MultiPromptChain` This is a specific type of chain that is used when routing between multiple different template, this help you route between any type of chain

`LLMRouterChain` This uses an LLM model itlsef to route between different subchains

`RouterOutputParser` This formates the LLM output to a dictionay which can be used for downstream processing like to determine which chain to use and what the input to that chain should be.

In [ ]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model = "zephyr",temperature = 0) # format = 'json' is also an argument 

In [ ]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [ ]:
chain.run("What is black body radiation?")

In [ ]:
chain.run("what is 2 + 2")

In [ ]:
chain.run("Why does every cell in our body contain DNA?")

Reminder: Download your notebook to you local computer to save your work.